# The closest block that will get the driver the highest tips based on the time of the day and day of the week?

## Preamble

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

## Load Data

In [9]:
df = pd.read_csv("datasets/yellow_tripdata_2013-01.csv")

## Preprocess data

In [10]:
# add day of week column
timestamp = pd.to_datetime(pd.Series(df['pickup_datetime']))
df['day_of_week'] = timestamp.dt.weekday_name
df['pickup_hour'] = timestamp.dt.hour
df['tip_percentage'] = df.tip_amount / df.trip_distance
df.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,day_of_week,pickup_hour,tip_percentage
0,CMT,2013-01-01 15:11:48,2013-01-01 15:18:10,4,1.0,-73.978165,40.757977,1,N,-73.989840,...,CSH,6.5,0.0,0.5,0.0,0.0,7.0,Tuesday,15,0.0
1,CMT,2013-01-06 00:18:35,2013-01-06 00:22:54,1,1.5,-74.006680,40.731781,1,N,-73.994499,...,CSH,6.0,0.5,0.5,0.0,0.0,7.0,Sunday,0,0.0
2,CMT,2013-01-05 18:49:41,2013-01-05 18:54:23,1,1.1,-74.004711,40.737770,1,N,-74.009831,...,CSH,5.5,1.0,0.5,0.0,0.0,7.0,Saturday,18,0.0
3,CMT,2013-01-07 23:54:15,2013-01-07 23:58:20,2,0.7,-73.974600,40.759945,1,N,-73.984737,...,CSH,5.0,0.5,0.5,0.0,0.0,6.0,Monday,23,0.0
4,CMT,2013-01-07 23:25:03,2013-01-07 23:34:24,1,2.1,-73.976252,40.748528,1,N,-74.002583,...,CSH,9.5,0.5,0.5,0.0,0.0,10.5,Monday,23,0.0


In [11]:
#drop unnecessary columns
df = df.drop(['vendor_id','passenger_count','trip_distance','rate_code',
              'store_and_fwd_flag','payment_type','fare_amount','surcharge','mta_tax',
             'tip_amount','tolls_amount','total_amount','dropoff_datetime',
              'dropoff_longitude','dropoff_latitude'], axis=1)
df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,day_of_week,pickup_hour,tip_percentage
0,2013-01-01 15:11:48,-73.978165,40.757977,Tuesday,15,0.0
1,2013-01-06 00:18:35,-74.006680,40.731781,Sunday,0,0.0
2,2013-01-05 18:49:41,-74.004711,40.737770,Saturday,18,0.0
3,2013-01-07 23:54:15,-73.974600,40.759945,Monday,23,0.0
4,2013-01-07 23:25:03,-73.976252,40.748528,Monday,23,0.0


In [5]:
def mileToDegree(mile):
    return float("{:.6f}".format(mile/60))

def getMaxLocation(location, mile):
    degree = mileToDegree(mile)
    max_locations = []
    max_locations.append({'longitude': location['longitude'] - degree,
                          'latitude': location['latitude'] - degree,})
    max_locations.append({'longitude': location['longitude'] + degree,
                          'latitude': location['latitude'] + degree,})
    return max_locations

inputs = {'mile': 1,
          'driver_location': {'longitude': -73.991340,'latitude': 40.744712},
          'hour': 16, # 4pm
          'day_of_week': 'Monday'}

max_location = getMaxLocation(inputs['driver_location'], inputs['mile'])
print(max_location)

df = df[(df['pickup_latitude'] > max_location[0]['latitude']) & 
        (df['pickup_longitude'] > max_location[0]['longitude']) &
        (df['pickup_latitude'] < max_location[1]['latitude']) & 
        (df['pickup_longitude'] < max_location[1]['longitude']) &
        (df['pickup_hour'] == inputs['hour']) & 
        (df['day_of_week'] == inputs['day_of_week'])]

df.head()

[{'longitude': -74.00800699999999, 'latitude': 40.728045}, {'longitude': -73.974673, 'latitude': 40.761379}]


,pickup_datetime,pickup_longitude,pickup_latitude,day_of_week,pickup_hour
1401,2013-01-07 16:31:19,-73.990359,40.749484,Monday,16
1409,2013-01-07 16:33:18,-73.978652,40.752362,Monday,16
236679,2013-01-14 16:03:00,-73.998682,40.734862,Monday,16
236682,2013-01-14 16:00:00,-73.983912,40.760037,Monday,16
236687,2013-01-14 16:00:00,-73.978950,40.753150,Monday,16


In [17]:
import matplotlib.path as mplPath

def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        z = mplPath.Path(np.array(zones.geometry[idx].exterior))
        if z.contains_point(np.array(p)):
            return zones['OBJECTID'][idx]
    return -1


def mapToZone(location):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones('datasets/block-groups-polygons.geojson')
    if (location['pickup_longitude'] and location['pickup_latitude']):
        pickup_location  = geom.Point(proj(float(location['pickup_longitude']), float(location['pickup_latitude'])))
        pickup_block = findBlock(pickup_location, index, zones)
        return pickup_block
    else: 
        return -1

# df['block_id'] = df.apply(lambda row: mapToZone(row), axis=1)
# df.head()